In [ ]:
import pandas as pd
dataset_siddipet=pd.read_csv('/content/siddipet_dataset_final.csv')

In [ ]:
dataset_siddipet.head()

,District,Mandal,Date,temp_max (⁰C)
0,Siddipet,Cheriyal,1/1/2018,31.6
1,Siddipet,Cheriyal,2/1/2018,32.2
2,Siddipet,Cheriyal,3/1/2018,30.2
3,Siddipet,Cheriyal,4/1/2018,29.7
4,Siddipet,Cheriyal,5/1/2018,27.9


In [ ]:
dataset_siddipet[10:]

,District,Mandal,Date,temp_max (⁰C)
10,Siddipet,Cheriyal,11/1/2018,28.9
11,Siddipet,Cheriyal,12/1/2018,32.6
12,Siddipet,Cheriyal,13/01/18,32.1
13,Siddipet,Cheriyal,14/01/18,31.4
14,Siddipet,Cheriyal,15/01/18,30.8
...,...,...,...,...
44575,Siddipet,Wargal,26-Sep-22,32.9
44576,Siddipet,Wargal,27-Sep-22,34.0
44577,Siddipet,Wargal,28-Sep-22,32.9
44578,Siddipet,Wargal,29-Sep-22,32.5


In [ ]:


dataset_siddipet['Date'] = pd.to_datetime(dataset_siddipet['Date'])

In [ ]:
dataset_siddipet[10:]

,District,Mandal,Date,temp_max (⁰C)
10,Siddipet,Cheriyal,2018-11-01,28.9
11,Siddipet,Cheriyal,2018-12-01,32.6
12,Siddipet,Cheriyal,2018-01-13,32.1
13,Siddipet,Cheriyal,2018-01-14,31.4
14,Siddipet,Cheriyal,2018-01-15,30.8
...,...,...,...,...
44575,Siddipet,Wargal,2022-09-26,32.9
44576,Siddipet,Wargal,2022-09-27,34.0
44577,Siddipet,Wargal,2022-09-28,32.9
44578,Siddipet,Wargal,2022-09-29,32.5


In [ ]:
dataset_siddipet['Date'] =dataset_siddipet['Date'].dt.date

In [ ]:
dataset_siddipet.shape

(44580, 4)

In [ ]:
avg_by_date = dataset_siddipet.groupby('Date')['temp_max (⁰C)'].mean()

In [ ]:
avg_by_date=avg_by_date.to_frame()

In [ ]:
avg_by_date.shape

(1613, 1)

In [ ]:
avg_by_date.head()

,temp_max (⁰C)
Date,
2018-01-01,31.125714
2018-01-02,31.971429
2018-01-03,36.280000
2018-01-04,37.365714
2018-01-05,42.777143


In [ ]:
avg_by_date=avg_by_date.reset_index()

In [ ]:
avg_by_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1613 entries, 0 to 1612
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1613 non-null   object 
 1   temp_max (⁰C)  1613 non-null   float64
dtypes: float64(1), object(1)
memory usage: 25.3+ KB


In [ ]:

avg_by_date['Date'] = pd.to_datetime(avg_by_date['Date'])


In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(result):
  ndt=adfuller(result,autolag='AIC')
  print("1 ADF :",ndt[0])
  print("2 p val:",ndt[1])
  print("3 lags",ndt[2])
  print("4 no. of observations",ndt[3])
  print("5 critical value")
  for key,val in ndt[4].items():
    print("\t",key," ",val)
  

In [ ]:
!pip install pmdarima

In [ ]:
!pip install pyfiglet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 KB 31.1 MB/s eta 0:00:00


In [ ]:
from pmdarima import auto_arima
import statsmodels.api as sm
import pyfiglet

dataset=[]

In [ ]:
# create a list of dataframes, one for each month
dfs = []
for i in range(1, 13):
    mask = (avg_by_date['Date'].dt.month == i)
    dfs.append(avg_by_date.loc[mask])

# print the resulting dataframes
for i, month_df in enumerate(dfs):
    adf_test(month_df['temp_max (⁰C)'])
    arimadata=auto_arima(month_df['temp_max (⁰C)'],trace=True)
    train=month_df.iloc[:-30]
    test=month_df.iloc[-30:]
    model=sm.tsa.arima.ARIMA(train['temp_max (⁰C)'],order=(1,0,5))
    model=model.fit()
    s=len(train)
    e=len(train)+len(test)-1
    pred=model.predict(start=s,end=e,typ='levels').rename('ARIMA Predictions')
    dataset.append(pred)
    

1 ADF : -3.860580584544115
2 p val: 0.002344885990468065
3 lags 0
4 no. of observations 123
5 critical value
	 1%   -3.4846672514209773
	 5%   -2.8853397507076006
	 10%   -2.5794629869786503
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=1.11 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=500.884, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=502.805, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=502.750, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=498.884, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.24 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.523 seconds


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


1 ADF : -2.891982025010319
2 p val: 0.046288354988110285
3 lags 0
4 no. of observations 112
5 critical value
	 1%   -3.4901313156261384
	 5%   -2.8877122815688776
	 10%   -2.5807296460459184
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=441.817, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=443.119, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=442.983, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=439.962, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=441.977, Time=0.12 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.716 seconds


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer in

1 ADF : -2.2655378274740854
2 p val: 0.1833236461645198
3 lags 4
4 no. of observations 119
5 critical value
	 1%   -3.4865346059036564
	 5%   -2.8861509858476264
	 10%   -2.579896092790057
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=490.169, Time=0.44 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=554.396, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=499.420, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=522.279, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1239.108, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=488.455, Time=0.40 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=490.632, Time=0.10 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=499.947, Time=0.30 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=490.779, Time=0.50 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=487.274, Time=0.11 sec
 ARIMA(0,0,4)(0,0,0)[0] intercept   : AIC=489.162, Time=0.12 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=490.883, T

...

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zero

1 ADF : -5.466681500003594
2 p val: 2.449964027529016e-06
3 lags 1
4 no. of observations 118
5 critical value
	 1%   -3.4870216863700767
	 5%   -2.8863625166643136
	 10%   -2.580009026141913
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=457.423, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=459.385, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=459.337, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=455.444, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.840 seconds


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1 ADF : -4.968157147635507
2 p val: 2.5649313438455954e-05
3 lags 13
4 no. of observations 110
5 critical value
	 1%   -3.4912451337340342
	 5%   -2.8881954545454547
	 10%   -2.5809876033057852
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=511.491, Time=0.48 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=581.172, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=516.650, Time=0.13 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=538.327, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1194.945, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=511.927, Time=0.29 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=518.034, Time=0.31 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=518.313, Time=0.73 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=513.176, Time=0.61 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=518.248, Time=0.20 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=513.193, Time=0.91 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=514.5

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting 

In [ ]:
len(dataset)

12

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [ ]:
for i in range(0,12):
  dataset[i]=dataset[i].to_frame()

In [ ]:
dates = pd.date_range(start='2023-01-01', end='2023-01-30', freq='D')
dataset[0]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-02-01', end='2023-02-28', freq='D')
# Drop the last two rows of the DataFrame
dataset[1] = dataset[1].drop(index=dataset[1].index[-2:])
dataset[1]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-03-01', end='2023-03-30', freq='D')
dataset[2]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-04-01', end='2023-04-30', freq='D')
dataset[3]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-05-01', end='2023-05-30', freq='D')
dataset[4]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-06-01', end='2023-06-30', freq='D')
dataset[5]['Dates']=dates


In [ ]:
dates = pd.date_range(start='2023-07-01', end='2023-07-30', freq='D')
dataset[6]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-08-01', end='2023-08-30', freq='D')
dataset[7]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-09-01', end='2023-09-30', freq='D')
dataset[8]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-10-01', end='2023-10-30', freq='D')
dataset[9]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-11-01', end='2023-11-30', freq='D')
dataset[10]['Dates']=dates

In [ ]:
dates = pd.date_range(start='2023-12-01', end='2023-12-30', freq='D')
dataset[11]['Dates']=dates

In [ ]:
dataset[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 94 to 123
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ARIMA Predictions  30 non-null     float64       
 1   Dates              30 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 612.0 bytes


The India Meteorological Department (IMD) declares a heatwave when the maximum temperature in a particular location reaches or exceeds a threshold value, which varies depending on the location. The threshold values for a heatwave are typically defined as temperatures that are significantly higher than the average maximum temperature for that location during the summer season.

In [ ]:
mean_temp=dataset[4]['ARIMA Predictions'].mean()
print(mean_temp)

39.20472015403835


In [ ]:
mean_temp=dataset[3]['ARIMA Predictions'].mean()
print(mean_temp)

37.87149008253771


In [ ]:
normal_temp =(37.87149008253771+39.20472015403835)/2  # Mean maximum temperature in Celsius for the hottest month

In [ ]:
print(normal_temp)

38.53810511828803


In [ ]:
for i in range(0,12):
  for index, row in dataset[i].iterrows():
    diff = row['ARIMA Predictions'] - normal_temp   
    if row['ARIMA Predictions'] >= 40:
       print("Severe Heat Wave on {} when temperature is {}".format(row['Dates'].date(),row['ARIMA Predictions']))
    elif row['ARIMA Predictions'] >= 38.5:
        print("Heat Wave on {} when temperature is {}".format(row['Dates'].date(),row['ARIMA Predictions']))
        


Heat Wave on 2023-04-01 when temperature is 38.918605680881655
Heat Wave on 2023-04-02 when temperature is 38.63441291132942
Severe Heat Wave on 2023-05-01 when temperature is 40.32974260086235
Severe Heat Wave on 2023-05-02 when temperature is 40.83743722479131
Severe Heat Wave on 2023-05-03 when temperature is 40.139078723289686
Heat Wave on 2023-05-04 when temperature is 39.89229413872982
Heat Wave on 2023-05-05 when temperature is 39.45132744070389
Heat Wave on 2023-05-06 when temperature is 39.398466166907475
Heat Wave on 2023-05-07 when temperature is 39.349057444743295
Heat Wave on 2023-05-08 when temperature is 39.302875776187605
Heat Wave on 2023-05-09 when temperature is 39.259710391268314
Heat Wave on 2023-05-10 when temperature is 39.219364286125234
Heat Wave on 2023-05-11 when temperature is 39.181653323897976
Heat Wave on 2023-05-12 when temperature is 39.146405394337926
Heat Wave on 2023-05-13 when temperature is 39.11345962830886
Heat Wave on 2023-05-14 when temperature